In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skylark.utils import logger

import pickle
import json
import re
from pathlib import Path

data_dir = Path(globals()["_dh"][0]).parent / "data"
throughput_dir = data_dir / "throughput" / "iperf3"
figure_dir = data_dir / "figures"
figure_dir.mkdir(exist_ok=True)

plt.style.use("seaborn-bright")
plt.set_cmap("plasma")

# list data files, sorted by date
data_files = sorted(throughput_dir.glob("*.json"), key=lambda x: x.name)
file = data_files[0]

In [ ]:
with Path("/home/ubuntu/skylark/data/throughput/iperf3/throughput_2022-01-04_22-12-49.json").open("rb") as f:
    data = json.load(f)
df = pd.DataFrame(data).set_index(["src", "dst"])
df["throughput_gbps"] = df["throughput_mbps"] / 1024
df

In [ ]:
def plot_grid(df, title="Throughput (Gbps)", key="throughput_gbps"):
    regions = sorted(set(df.index.get_level_values("src")))
    data_grid = np.zeros((len(regions), len(regions)))

    for i, src in enumerate(regions):
        for j, dst in enumerate(regions):
            try:
                data_grid[i, j] = df.loc[src, dst][key]
            except KeyError:
                data_grid[i, j] = np.nan

    fig, ax = plt.subplots(1, 1, figsize=(9, 9))
    ax.imshow(data_grid)
    ax.set_title(title)
    ax.set_xticks(np.arange(len(regions)))
    ax.set_yticks(np.arange(len(regions)))
    ax.set_xticklabels(regions)
    ax.set_yticklabels(regions)

    for tick in ax.get_xticklabels():
        tick.set_rotation(90)

    # compute mean point of non nan values
    mean_point = np.nanmean(data_grid)
    for i, row in enumerate(data_grid):
        for j, col in enumerate(row):
            if not np.isnan(col):
                ax.text(j, i, round(col, 1), ha="center", va="center", color="white" if col < mean_point else "black")

    fig.patch.set_facecolor("white")
    fig.subplots_adjust(hspace=0.6)
    ax.figure.colorbar(ax.images[0], ax=ax)
    return fig, ax


df_plot = df.copy()
fig, _ = plot_grid(df_plot, title="Throughput (Gbps)", key="throughput_gbps")
fig.savefig(figure_dir / "udp_throughput_grid.pdf", bbox_inches="tight")
fig, _ = plot_grid(df_plot, title="Packet Loss (%)", key="loss_percent")
fig.savefig(figure_dir / "udp_packet_loss_grid.pdf", bbox_inches="tight")